In [ ]:
import numpy as np
import scipy as scp
import pylab as pyl
import pywt
import pandas as pd
import holoviews as hv
import param
import panel as pn
from panel.pane import LaTeX
hv.extension('bokeh')
import warnings
warnings.filterwarnings('ignore')
from PIL import Image
from io import BytesIO
import requests
import time
import itertools
import hvplot.pandas
from bokeh.models import HoverTool

# Nesterov and Gradient Descent.

The following scheme compute the trajectory of the solution of 
\begin{equation}
\ddot{x}(t)+\frac{\alpha}{t}\dot{x}(t)+\nabla F(x(t))=0
\end{equation}
for $F(x)=\frac{\mu}{2}x^2$ with initial condition $t_0=0$ and $x(t_0)=x_0$ and $\dot{x}(t_0)=0$

In [ ]:
def nesterov(x0,h,T,alpha,mu):
    t0=0
    step=np.sqrt(h)
    nb=int(np.floor(T)/step)
    x=np.zeros(nb)
    x[0]=x0
    x[1]=x0-h*2*x[0]
    y=np.copy(x)
    c=1-mu*h
    for k in range(1,nb-1):
        y[k+1]=x[k]+(k/(k+alpha))*(x[k]-x[k-1])
        #x[k+1]=y[k+1]-mu*h*y[k+1]
        x[k+1]=c*y[k+1]
    temps=np.linspace(t0,t0+T,nb)
    f=mu*x**2/2*temps**alpha
    #stralpha="%2.2f" % alpha
    #lab='alpha= '+stralpha
    xnes=hv.Curve((temps,x),kdims='times',vdims='iterates')
    fnes=hv.Curve((temps,f),kdims='times',vdims='rescaled_values')
    return xnes,fnes

In [ ]:
def nesterovrescale(x0,h,alpha,mu,T):
    t0=0
    #T=2*alpha+2
    step=np.sqrt(h)
    nb=int(np.floor(T)/step)
    x=np.zeros(nb)
    x[0]=x0
    x[1]=x0-h*2*x[0]
    y=np.copy(x)
    xr=np.copy(x)
    c=1-mu*h
    for k in range(1,nb-1):
        y[k+1]=x[k]+(k/(k+alpha))*(x[k]-x[k-1])
        x[k+1]=c*y[k+1]
    temps=np.linspace(t0,t0+T,nb)
    tempsrescale=temps**2/(2*(alpha+1))
    xnes=hv.Curve((tempsrescale,x),kdims='times',vdims='iterates',label='Nesterov Rescaled')

    return xnes

In [ ]:
def gradientmux2(x0,step,T,mu):
    t0=0
    nb=int(np.floor(T)/step)
    temps=np.linspace(t0,t0+T)
    x=x0*np.exp(-mu*temps)
    x_grad=hv.Curve((temps,x),kdims='times',vdims='iterates',label='Gradient')
    fx_grad=hv.Curve((temps,(mu*x**2)/2),kdims='times',vdims='iterates')
    return x_grad,fx_grad

In [ ]:
x0=1
mu=1
alpha=20
T=5
h=0.0000001
step=np.sqrt(h)
options = dict(width=800,height=250,toolbar=None)
xnes=nesterovrescale(x0,h,alpha,mu,np.sqrt(2*(alpha+1)*T))
#xnes
x_grad,fx_grad=gradientmux2(x0,h,T,mu)
pn.Column(xnes[0:T].opts(**options)*x_grad.opts(**options))

In [ ]:
class Nesterov_Rescaled(param.Parameterized):
    alpha = param.Number(5,bounds=(3,500))
    logh = param.Number(-7,bounds=(-10,0))
    T = param.Number(3,bounds=(1,10))
    def view(self):
        x0=1
        mu=1
        h=10**self.logh
        T2=np.sqrt(2*(self.alpha+1)*self.T)
        xnes=nesterovrescale(x0,h,self.alpha,mu,T2)
        x_grad,fx_grad=gradientmux2(x0,h,T,mu)
        options = dict(width=670,height=250,toolbar=None)
        return hv.Curve(xnes[0:self.T]).opts(**options)*x_grad[0:self.T].opts(**options)

In the Following cell we can compare both solutions of 
\begin{equation}\label{Grad}\tag{Grad}
\dot{x}(t)+\nabla F(x(t))=0
\end{equation}
and of 
\begin{equation}\label{Nes}\tag{Nes}
\ddot{x}(t)+\frac{\alpha}{t}\dot{x}(t)+\nabla F(x(t))=0
\end{equation}
when $F(x)=\frac{\mu}{2}||x||^2$.
More precisely we compute 
\begin{enumerate}
\item In red, we display the exact solution of \eqref{Grad}
\item In blue we display $y(\sqrt{2(\alpha+1)t})$ where $y$ is a solution of \eqref{Nes}. 
\end{enumerate}
The solution of \eqref{Nes} is computed using the discret Nesterov scheme with a step $h=10^{logh}$ which can changed in the applet. Initial conditions are set to $t_0=0$, $x(t_0)=1$ for both equations and $\dot{x}(t_0)=0$ for \eqref{Nes}. 
The time $T$ and the value of $\alpha$ can be changed in the applet.

In [ ]:
NesR= Nesterov_Rescaled()
pn.Row(NesR.param,NesR.view)